Usage

  cmake [options] <path-to-source>
  cmake [options] <path-to-existing-build>
  cmake [options] -S <path-to-source> -B <path-to-build>

Specify a source directory to (re-)generate a build system for it in the
current working directory.  Specify an existing build directory to
re-generate its build system.

Run 'cmake --help' for more information.



In [5]:
# !pip install openai
# !pip install gradio typing-extensions
# !pip install pyopenjtalk

HE
CMake is in the system path. Version: cmake version 3.27.3

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [29]:
import os
import openai 
import gradio as gr


import json 
from tqdm import tqdm 
import time
from datetime import datetime

import concurrent.futures


# from newsplease import NewsPlease
from pathlib import Path

In [30]:
openai_secret = None
with open("../openai_chat_secret.txt", 'r') as f1:
    openai_secret = f1.read()
openai.api_key = openai_secret

In [35]:
AI_CHARACTER_NAME = "Shiori"
USER_NAME = "Tangerine"

In [91]:
personality = f"""
You now are {AI_CHARACTER_NAME}. Being a caster, She can be somewhat clueless, naive, and aloof when needed. She has a fondness for books. 
The current person you are talking to is {USER_NAME}.
"""

In [37]:
def build_conversation(personality, history_chat, current_chat):
    conversaition = "role setting:" + personality + ".\n" \
                    + "history:" + history_chat + ".\n" \
                    + "current line:" + current_chat + ".\n"
    return conversaition

In [92]:

def speak_to_ai_waifu(conversation):
    messages = []
    messages.append({'role': 'assistant', 'content':  conversation})
    # print(message)
    
    # response = openai.Completion.create(
    #   model="davinci",
    #   prompt=conversation,
    #   # temperature=1,
    #   # max_tokens=500,
    #   # top_p=0.7,
    #   # frequency_penalty=0.0,
    #   # presence_penalty=0.0,
    #   # stop=["\n"]
    # )
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      # model="davinci",
      messages=messages,
      # temperature=1,
      # max_tokens=500,
      # top_p=0.7,
      # frequency_penalty=0.0,
      # presence_penalty=0.0,
      # stop=["\n"]
    )
    
    return response

In [93]:
# conversation = build_conversation(personality=personality, history_chat="", current_chat="")
# respond = speak_to_ai_waifu(conversation)
# print(respond)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Shiori sat in a cozy corner of the library, absorbed in a book about ancient magical artifacts. The warm light from the nearby window bathed her in a soft glow, adding to the serene atmosphere. She absentmindedly twirled a strand of her long, silver hair around her finger as she read, lost in the world of words and knowledge.\n\nSuddenly, a familiar voice called out to her from across the room. Shiori looked up, her lavender eyes shimmering with curiosity. It was Tangerine, a fellow mage she had befriended during their studies at the Academy of Arcane Arts. As Tangerine approached, Shiori closed her book and smiled warmly.\n\n\"Hey, Tangerine! What brings you to the library?\" Shiori said, her voice filled with genuine joy. She was always happy to see her friends and take a break from her solitary reading adventures.\n\nTangerine chuckled and leaned against the bookshelf next 

In [42]:
# respond_words = respond['choices'][0]['message']['content'] 

# history = f""" {AI_CHARACTER_NAME}: respond_words """


In [47]:
def build_history_chat(history_chat_buffer, num_chat):
    history_chat = []
    len_buffer = len(history_chat_buffer)
    for i in range(len_buffer - min(num_chat,len_buffer), len_buffer):
        history_chat.append(history_chat_buffer[i][0] + ":" + history_chat_buffer[i][1])
    return "\n".join(history_chat)

In [75]:
def format_history_chat(history_chat_buffer):
    formatted_text = ""
    for x in history_chat_buffer:
        formatted_text += "\n".join(["      "+x2.strip()+"." for x2 in x[1].strip().split(".")])
        formatted_text += "\n"
    return formatted_text

In [87]:
history_chat_buffer = []
NUM_HISTORY_CHAT_SEND_To_AI = 5

In [94]:
def text_chat(user_inputs):
    ### User input
    # user_inputs = input('Say anything:')
    print(user_inputs)

    ### Build history chat, getting last 5 chat from history buffer
    history_chat = build_history_chat(history_chat_buffer, NUM_HISTORY_CHAT_SEND_To_AI)

    ### Build conversation
    conversation = build_conversation(personality=personality, history_chat=history_chat, current_chat=user_inputs)
    
    ### Send conversation to AI waifu
    respond = speak_to_ai_waifu(conversation)

    ### Get respond from AI waifu
    responding_words = respond['choices'][0]['message']['content'] 
    # print(f"{AI_WAIFU_NAME}: {responding_words}")

    ### save conversation to history buffer
    history_chat_buffer.append([USER_NAME,user_inputs])
    history_chat_buffer.append([AI_WAIFU_NAME,responding_words])


    return format_history_chat(history_chat_buffer)


In [98]:
# text_chat("that's interesting.")

In [99]:
from transformers import pipeline

import gradio as gr

# asr = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
# classifier = pipeline("text-classification")


# def speech_to_text(speech):
#     text = asr(speech)["text"]
#     return text


# def text_to_sentiment(text):
#     return classifier(text)[0]["label"]


buffer = []

demo = gr.Blocks()

with demo:
    # audio_file = gr.Audio(type="filepath")
    text = gr.Textbox(lines=3, placeholder="Enter your text here...")
    # label = gr.Label()
    text_history = gr.Textbox(lines=5)


    # b1 = gr.Button("Recognize Speech")
    b2 = gr.Button("Text Chat with Waifu")

    # b1.click(speech_to_text, inputs=audio_file, outputs=text)
    b2.click(text_chat, inputs=text, outputs=text_history)
    print(buffer)
    text_history = buffer

demo.launch()

[]
Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


In [96]:
# # print(history_chat_buffer)
# for x in history_chat_buffer:
#     print(x[0]+":")
#     print("\n".join(["      "+x2.strip()+"." for x2 in x[1].strip().split(".")]))

In [97]:
# # print(history_chat_buffer)
# for x in history_chat_buffer:
#     print(x[0]+":", x[1])
